In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install --upgrade moviepy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 68.1 MB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=6d207401af51f908702617d7c146ac58a7a85ec2a2e4ef8f7b786d2e6ebbc35a
  Stored in directory: /root/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy
  Attempting uninstall: moviepy
    Found existing installation: moviepy 0.2.3.5
    Uninstalling moviepy-0.2.3.5:
      Successfully uninstalled moviepy-0.2.3.5


In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pylab
from google.colab.patches import cv2_imshow
from moviepy.editor import VideoFileClip
from moviepy import *
import moviepy
%matplotlib inline
import time
import os
import glob

### Load weights and cfg

In [ ]:
weights_path =os.path.join("/content/drive/MyDrive/project_data/yolov3-tiny.weights")
cfg_path =os.path.join("/content/drive/MyDrive/project_data/tiny.cfg.txt")
print(" weights and cfg are loaded")

 weights and cfg are loaded


### load neural net in cv2 

In [ ]:
net=cv2.dnn.readNetFromDarknet(cfg_path ,weights_path)

### Get layers name

In [ ]:
names=net.getLayerNames()

In [ ]:
layers_names=[names[i - 1] for i in net.getUnconnectedOutLayers()]

### define coco dataset labels

In [ ]:
labels_path=os.path.join("/content/drive/MyDrive/project_data/coco.names.txt")
labels = open(labels_path).read().strip().split("\n")

### define the pipeline on each image 

In [ ]:
def process(image):
  (H,W)=image.shape[:2]
  blob= cv2.dnn.blobFromImage(image,1/255.0,(416,416),crop=False,swapRB=False)
  net.setInput(blob)
  layers_output=net.forward(layers_names)
  boxes=[]
  confidences=[]
  classIDs=[]
  for output in layers_output:
    for detection in output:
        scores=detection[5:]
        classID=np.argmax(scores)
        confidence=scores[classID]
        if classID==0 :
          if (confidence >0.3):
              box=detection[:4] * np.array([W,H,W,H])
              bx,by,bw,bh=box.astype("int")
            
              x=int(bx-(bw/2))
              y=int(by-(bh/2))
           
              boxes.append([x,y,int(bw),int(bh)])
              confidences.append(float(confidence))
              classIDs.append(classID)
    idx=cv2.dnn.NMSBoxes(boxes,confidences,0.3,0.4)
    if (len(idx)>0):
      for i in idx.flatten():
        (x,y)=[boxes[i][0],boxes[i][1]]
        (w,h)=[boxes[i][2],boxes[i][3]]
        if bh>bw :
          cv2.rectangle (image,(x,y),(x+w ,y+h),(0,255,0),2)
        else :
          cv2.rectangle (image,(x,y),(x+w ,y+h),(255,0,0),2)
          cv2.putText(image,("fall detected"),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,2,(255,0,0),2)
        
  return image

In [ ]:
white_output = ("old_detected4.mp4")
clip1 = VideoFileClip("old_trimmed3.mp4") 
white_clip = clip1.fl_image(process)
white_clip.write_videofile(white_output, audio=False)

Moviepy - Building video old_detected4.mp4.
Moviepy - Writing video old_detected4.mp4



t: 100%|██████████| 197/197 [00:36<00:00,  6.54it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.8/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file old_trimmed3.mp4, 2764800 bytes wanted but 0 bytes read,at frame 196/197, at time 6.54/6.54 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready old_detected4.mp4
